In [4]:
import gymnasium as gym
import torch
import random
from collections import namedtuple, deque


**Replay Memory**

In [5]:
Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))

In [6]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = deque([], maxlen = self.capacity) # cyclic buffer of bounded size
    
    def __len__(self):
        return len(self.memory)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
        

**DQN**

In [7]:
class DQN(torch.nn.Module):
    
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(n_observations, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256,256), # only one hidden layer, the first two regarding the convolution as processing of input images are dropped
            torch.nn.ReLU(),
            torch.nn.Linear(256, n_actions)
        )
    
    def forward(self, x):
        return self.layers(x)
        

***Environment***

In [8]:
env = gym.make("MountainCar-v0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.99
EPS_END = 0.001
EPS_DECAY = 0.9
REPLAY_MEMORY = 100000
LR = 0.0005 #1e-4


state, info = env.reset()
n_actions = env.action_space.n
n_observations = len(state)

memory = ReplayMemory(REPLAY_MEMORY)

estimate_network = DQN(n_observations, n_actions)
target_network = DQN(n_observations, n_actions)


def act(state, eps):
    if random.random() < eps:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
    else: 
        with torch.no_grad(): # improves computation
            return estimate_network(state).max(1).indices.view(1,1)
        